# Création du jeu de données

Données recherchées :
- Positions et vélocités des planètes par rapport au barycentre du système solaire, sur une période la plus longue possible (ça varie d'une planète à l'autre).
- Masse de chaque planète.

| ID | Name	| Start of Interval	| End of Interval |
|----|------|-------------------|-----------------|
| 10 | Sun [Sol] | B.C. 9999-03-20 00:00 |	A.D. 9999-12-31 12:00 |
| 199 |	Mercury	| B.C. 9999-03-20 00:00	| A.D. 9999-12-31 12:00 |
| 299 |	Venus	| B.C. 9999-03-20 00:00	| A.D. 9999-12-31 12:00 |
| 399 |	Earth	| B.C. 9999-03-20 00:00	| A.D. 9999-12-31 12:00 |
| 499 |	Mars	| 1600-01-01 00:00	| 2600-01-02 00:00 |
| 599 |	Jupiter	| 1600-01-10 00:00 (1600-01-11)	| 2200-01-10 00:00 (2200-01-09) |
| 699 |	Saturn	| 1749-12-30 00:00 (1749-12-31)	| 2250-01-06 00:00 |
| 799 |	Uranus	| 1600-01-04 00:00	| 2399-12-17 00:00 |
| 899 |	Neptune	| 1600-01-04 00:00	| 2400-01-02 00:00 |

## 1. Importation des librairies

In [3]:
import pandas as pd
from astroquery.jplhorizons import Horizons
from pandas import DataFrame
from datetime import datetime
from dateutil.relativedelta import relativedelta

## 2. Définition des planètes cibles et des intervalles de temps pour lesquels récupérer les données

In [4]:
planets: dict[str, str] = {
    "Mercure": "199",
    "Venus":   "299",
    "Terre":   "399",
    "Mars":    "499",
    "Jupiter": "599",
    "Saturne": "699",
    "Uranus":  "799",
    "Neptune": "899",
    #"Pluton": "999",
}

date_format: str = "%Y-%m-%d"
min_date: str = "1749-12-31" # Date minimale à partir de laquelle on a des données pour toutes les planètes (Limité par Saturne)
max_date: datetime = datetime.strptime("2200-01-09", date_format) # Date maximale à partir de laquelle on a des données pour toutes les planètes (Limité par Jupiter)
time_step = '1d' # Intervalles de 1 jour

## 3. Récupération des données

In [ ]:
all_dataframes = []

for name, target in planets.items():
    # On va chercher les données par intervalles de 100 ans parce que sur l'intervalle complet, le nombre de lignes retournées dépasse la limite permise par l'API.
    for i in range(5):
        start_date = datetime.strptime(min_date, date_format) + relativedelta(years=i * 100)
        end_date = start_date + relativedelta(years=100, days=-1)
        if end_date > max_date:
            end_date = max_date
        start_date_str = datetime.strftime(start_date, date_format)
        end_date_str = datetime.strftime(end_date, date_format)

        horizons = Horizons(
            id=target,
            location='@0', # Barycentre du système solaire
            epochs={'start': start_date_str, 'stop': end_date_str, 'step': time_step}
        )
        vectors = horizons.vectors()

        df: DataFrame = vectors.to_pandas()

        df['planet_id'] = target
        df['planet_name'] = name

        columns_mask = ['planet_id', 'planet_name', 'datetime_jd', 'datetime_str','x','y','z','vx','vy','vz']

        df = df[columns_mask].copy()

        all_dataframes.append(df)

dataset = pd.concat(all_dataframes, ignore_index=True)

dataset.to_json('planet_coordinates_and_velocities_from_1749-12-31_to_2200-01-09.json', orient='records', lines=True)

print(dataset.sample(5))


       planet_id planet_name  datetime_jd                    datetime_str  \
638037       499        Mars    2505163.5  A.D. 2146-Oct-21 00:00:00.0000   

               x         y         z        vx        vy        vz  
638037  1.329803  0.491329 -0.021885 -0.004338  0.014331  0.000406  
